In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d

In [ ]:
data = pd.read_pickle('data/recordings/2021-09-18_22_35_56.328239.pkl')
data = pd.read_pickle('data/recordings/2021-09-19_14_59_39.406932.pkl')
data = pd.read_pickle('/tmp/2021-09-28_14_17_07.000342.pkl')
data = pd.read_pickle('./2021-09-28_14_59_58.491483.pkl')

In [ ]:
data.head()

In [ ]:
data['speed'] = data['velocity'].apply(np.linalg.norm)
data['time'] -= data['time'].iloc[0]
data['x_position'] = data['position'].apply(lambda pos: pos[0])
data['y_position'] = data['position'].apply(lambda pos: pos[1])

In [ ]:
plt.scatter(data['x_position'], data['y_position'], alpha=0.1)
plt.gca().set_aspect('equal')
plt.show();

In [ ]:
plt.plot(data['speed_actuator']);

In [ ]:
plt.plot(data['speed_actuator'].diff());

In [ ]:
plt.plot(data['speed']);

In [ ]:
plt.plot(data['speed_actuator'] - data['speed']);

In [ ]:
plt.plot(data['speed'].diff() / data['time'].diff());

In [ ]:
plt.plot(data['delta']);

In [ ]:
plt.plot(data['delta'].diff());

In [ ]:
start_idx = 2350
trajectory_len = 150

subdata = data.iloc[start_idx:(start_idx + trajectory_len)]

In [ ]:
plt.scatter(subdata['x_position'], subdata['y_position'], alpha=0.5)
plt.scatter(data['x_position'], data['y_position'], alpha=0.01)
plt.gca().set_aspect('equal')

plt.show();

plt.scatter(subdata['x_position'], subdata['y_position'], alpha=0.5);
plt.show()

In [ ]:
plt.plot(subdata['time'], 10 * subdata['delta'], label='10 x delta');
plt.plot(subdata['time'], subdata['speed_actuator'] - subdata['speed'], label='speed_actuator')
plt.legend()
plt.show()

In [ ]:
delta_fn = interp1d(subdata['time'], subdata['delta'], assume_sorted=True)
speed_act_fn = interp1d(subdata['time'], subdata['speed_actuator'], assume_sorted=True)

In [ ]:
start_time, end_time = subdata['time'].min(), subdata['time'].max()
horizon = end_time - start_time
steps = 20
time_steps = np.array([start_time + i * horizon / steps for i in range(steps)])
delta_steps = delta_fn(time_steps)
speed_act_steps = speed_act_fn(time_steps)
plt.plot(time_steps, 10 * delta_steps)
plt.plot(time_steps, speed_act_steps)
plt.plot(subdata['time'], 10 * subdata['delta']);
plt.plot(subdata['time'], subdata['speed_actuator']);

In [ ]:
delta_subfn = interp1d(time_steps, delta_steps, assume_sorted=True, fill_value='extrapolate')
speed_act_subfn = interp1d(time_steps, speed_act_steps, assume_sorted=True, fill_value='extrapolate')

In [ ]:
((delta_subfn(subdata['time']) - subdata['delta']) ** 2).mean() / subdata['delta'].abs().mean()

In [ ]:
((speed_act_subfn(subdata['time']) - subdata['speed_actuator']) ** 2).mean()

In [ ]:
((speed_act_subfn(subdata['time']) - subdata['speed_actuator']) ** 2).max()

# OK, so it seems 20 points should suffice, let's now check out the 2D interpolation

In [ ]:
plt.scatter(subdata['x_position'], subdata['y_position'], alpha=0.1);

In [ ]:
x_coord_fn = interp1d(subdata['time'], subdata['x_position'], assume_sorted=True)
y_coord_fn = interp1d(subdata['time'], subdata['y_position'], assume_sorted=True)

In [ ]:
x_coord_steps = x_coord_fn(time_steps)
y_coord_steps = y_coord_fn(time_steps)

In [ ]:
plt.scatter(subdata['x_position'], subdata['y_position'], alpha=0.1)
plt.scatter(x_coord_steps, y_coord_steps, alpha=0.8, color='k');

In [ ]:
x_coord_subfn = interp1d(time_steps, x_coord_steps, assume_sorted=True, fill_value='extrapolate')
y_coord_subfn = interp1d(time_steps, y_coord_steps, assume_sorted=True, fill_value='extrapolate')

In [ ]:
((x_coord_subfn(subdata['time']) - subdata['x_position']) ** 2).mean()

In [ ]:
((x_coord_subfn(subdata['time']) - subdata['x_position']) ** 2).max()

In [ ]:
((y_coord_subfn(subdata['time']) - subdata['y_position']) ** 2).mean()

In [ ]:
((x_coord_subfn(subdata['time']) - subdata['x_position']) ** 2).max()